## Build parameter files to reproject MOD05 by HEG command

command like: './subset_stitch_swath -p HegSwathStitch.prm_beichen'

In [ ]:
import os

In [ ]:
MOD05_L2_HDF_FOLDER = '/disk2/Data/MOD05_L2'
MOD05_L2_GEOTIFF_FOLDER = '/disk2/Data/MOD05_L2_GeoTiff'
HEGCLI_CMD_FOLDER = '/disk2/Data/MOD05_L2_HEG_cmd'

day_night_divide = 1.5*1024*1024

In [ ]:
cmd_file_template = '''
NUM_RUNS = 1

BEGIN
NUMBER_INPUTFILES = 7
INPUT_FILENAMES = [input_hdf_files]
OBJECT_NAME = mod05|
FIELD_NAME = Water_Vapor_Near_Infrared|
BAND_NUMBER = 1
SPATIAL_SUBSET_UL_CORNER = ( 50.0 120.0 )
SPATIAL_SUBSET_LR_CORNER = ( 20.0 150.0 )
OUTPUT_OBJECT_NAME = mod05|
OUTGRID_X_PIXELSIZE = 0.01
OUTGRID_Y_PIXELSIZE = 0.01
RESAMPLING_TYPE = NN
OUTPUT_PROJECTION_TYPE = GEO
ELLIPSOID_CODE = WGS84
OUTPUT_PROJECTION_PARAMETERS = ( 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0  )
OUTPUT_FILENAME = [output_tif_file]
SAVE_STITCHED_FILE = NO
OUTPUT_STITCHED_FILENAME = [output_hdf_file]
OUTPUT_TYPE = GEO
END

'''

cmd_template = '/home/beichen/software/opt/HEG_tool/bin/subset_stitch_swath -p [parameter_file];'

In [ ]:
cmd_list_str = ''
years = [2018, 2019]
for year in years:
    year_cmd_folder = os.path.join(HEGCLI_CMD_FOLDER, str(year))
    if not os.path.exists(year_cmd_folder):
        os.makedirs(year_cmd_folder)
    mod05_hdf_folder = os.path.join(MOD05_L2_HDF_FOLDER, str(year))
    mod05_year_filelist = os.listdir(mod05_hdf_folder)
    mod05_geo_folder = os.path.join(MOD05_L2_GEOTIFF_FOLDER, str(year))
    for day_idx in range(1,366,1):
        sub_timestr = 'A' + str(year) + (3-len(str(day_idx)))*'0' + str(day_idx)
        day_hdf_files = [hdf_file for hdf_file in mod05_year_filelist if sub_timestr in hdf_file]
        daytime_hdf_files = []
        for day_hdf_file in day_hdf_files:
            mod05_hdf_filename = os.path.join(mod05_hdf_folder, day_hdf_file)
            size_in_bytes = os.path.getsize(mod05_hdf_filename)
            if size_in_bytes > day_night_divide:
                daytime_hdf_files.append(mod05_hdf_filename)
        merge_hdf_filename = '|'.join(daytime_hdf_files)
        output_tif_filename = os.path.join(mod05_geo_folder, 'MOD05_L2.' + sub_timestr + '.tif')
        output_hdf_filename = os.path.join(mod05_geo_folder, 'MOD05_L2.' + sub_timestr + '.hdf')
        cmd_filename = os.path.join(year_cmd_folder, 'MOD05_L2_' + sub_timestr + '.prm_beichen')
        
        cmd_file_str = cmd_file_template.replace('[input_hdf_files]', merge_hdf_filename)
        cmd_file_str = cmd_file_str.replace('[output_tif_file]', output_tif_filename)
        cmd_file_str = cmd_file_str.replace('[output_hdf_file]', output_hdf_filename)
        
        with open(cmd_filename, 'w') as f:
            f.write(cmd_file_str)
            print(cmd_filename)
        
        cmd_c = cmd_template.replace('[parameter_file]', cmd_filename)
        cmd_list_str = cmd_list_str + cmd_c

with open(os.path.join(HEGCLI_CMD_FOLDER, 'HEGCLI_cmd.sh'), 'w') as f:
    f.write(cmd_list_str)
    print('over!')

# Just run .sh file can be works!